In [17]:
import re
import tweepy
from textblob import TextBlob

class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # keys and tokens from the Twitter Dev Console
        consumer_key = 'PalJpnUSeKS8quwt9pPWCwfGO'
        consumer_secret = 'Nohft5QBmfhJh4iBkGZKNJSKCeS2zvAQwFnWxCmz3g3Go3rATD'
        access_token = '1609194679962316800-okP4RTzGvoRfhrVe8PhDH9p9Rgt6ug'
        access_token_secret = 'kMKUxaB2feQqtx0XqKLymuMEPsKxDQFnIlSqg7nH9tukc'

        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth, wait_on_rate_limit=True)
        except Exception as e:
            print(f"Error: Authentication Failed - {e}")

    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'

    def get_tweets(self, query, count=10):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []

        try:
            # Use Cursor for better pagination and rate limiting
            # Note: Twitter API v1.1 search is deprecated, using v2 approach
            for tweet in tweepy.Cursor(self.api.search_tweets,
                                     q=query,
                                     lang="en",
                                     result_type="recent",
                                     tweet_mode="extended").items(count):

                # empty dictionary to store required params of a tweet
                parsed_tweet = {}

                # saving text of tweet (use full_text for extended tweets)
                parsed_tweet['text'] = tweet.full_text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.full_text)

                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)

            # return parsed tweets
            return tweets

        except tweepy.TweepyException as e:
            # print error (if any)
            print(f"Error: {str(e)}")
            return []

def main():
    # creating object of TwitterClient Class
    api = TwitterClient()
    # calling function to get tweets
    tweets = api.get_tweets(query='Donald Trump', count=200)

    if not tweets:
        print("No tweets retrieved. Please check your API credentials and query.")
        return

    # picking positive tweets from tweets
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
    # percentage of positive tweets
    print(f"Positive tweets percentage: {100*len(ptweets)/len(tweets):.1f}%")

    # picking negative tweets from tweets
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
    # percentage of negative tweets
    print(f"Negative tweets percentage: {100*len(ntweets)/len(tweets):.1f}%")

    # percentage of neutral tweets
    neutral_count = len(tweets) - (len(ntweets) + len(ptweets))
    print(f"Neutral tweets percentage: {100*neutral_count/len(tweets):.1f}%")

    # printing first 10 positive tweets
    print("\n\nPositive tweets:")
    for tweet in ptweets[:10]:
        print(f"- {tweet['text'][:100]}...")  # Truncate for readability

    # printing first 10 negative tweets
    print("\n\nNegative tweets:")
    for tweet in ntweets[:10]:
        print(f"- {tweet['text'][:100]}...")  # Truncate for readability

if __name__ == "__main__":
    # calling main function
    main()

Error: 403 Forbidden
453 - You currently have access to a subset of X API V2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.x.com/en/portal/product
No tweets retrieved. Please check your API credentials and query.
